## Todo List
- [x] Checar se formato de datas conferem.
- [x] Remover UGs sem liberação OC
- [x] Considerar coluna montagem. Onde estiver vazio, pegar valor de Torres para EOL, no skate.
- [ ] Separar um cojunto aleatório para teste
- [x] Calcular tempos de construção por marco: (IO_realizado - OC_realizado), (CC_realizado - OC_realizado) ...etc  (ordem:IO,CC,ME,OT)
- [x] Calcular atraso da operação comercial (OC_realizado - OC_outorgado)
- [ ] Calcular média e dp do tempo de construção.
- [ ] Calcular data dos marcos medios (OC_outorgado - tempo_medio_de_construcao) por fonte.
- [ ] Calcular atrasos com base nos marcos medios
- [ ] Calcular media e dp dos atrasos. 
- [ ] Normalizar os atrasos calculados (atraso - atraso_medio)/dp
- [ ] Construir matriz de correlação de Pearson
- [ ] Calcular os coeficientes para regressão linear
- [ ] Para UHE pegar todas UGs, para as demais pegar a primeira UG apenas 
- [ ] Comparar pegando primeira UG e media por grupo



In [1]:
import pandas as pd
import numpy as np
import re
import pyodbc
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats
from itertools import combinations

In [2]:
def outliers(df:pd.DataFrame,cols : list):
    index_outliers =  []
    for col in cols:
        s = df[col]
        Q1 = s.quantile(0.1)
        Q3 = s.quantile(0.9)
        lower_bound =  Q1
        upper_bound =  Q3 
        index_outliers.extend(s.index[(s < lower_bound) | (s > upper_bound)])
    return set(index_outliers)

In [3]:
bd_combined = pd.read_parquet("bd_combined.gzip")
bd_combined.IdeUsinaOutorga = bd_combined.IdeUsinaOutorga.astype(str)
mask = (bd_combined.SigTipoGeracao == "UHE") & (bd_combined.IdeUsinaOutorga.duplicated(keep=False))
bd_combined.loc[mask,"IdeUsinaOutorga"] = bd_combined.loc[mask,"IdeUsinaOutorga"] +"_"+ bd_combined.loc[mask].groupby("IdeUsinaOutorga").cumcount().astype(str)

In [4]:
bd_combined_OC = bd_combined[bd_combined.DatUGInicioOpComerOutorgado.notna()
                            & bd_combined.DatLiberOpComerRealizado.notna()].copy(deep=True)

In [42]:
calculos_OC_por_UG = bd_combined_OC[["IdeUsinaOutorga","SigTipoGeracao"]].copy(deep=True)
calculos_OC_por_UG["IO_OC"] = bd_combined_OC.DatLiberOpComerRealizado-  bd_combined_OC.DatInicioObraRealizado 
calculos_OC_por_UG["CC_OC"] = bd_combined_OC.DatLiberOpComerRealizado-  bd_combined_OC.DatConcretagemRealizado
calculos_OC_por_UG["ME_OC"] = bd_combined_OC.DatLiberOpComerRealizado-  bd_combined_OC.DatMontagemRealizado
calculos_OC_por_UG["MT_OC"] = bd_combined_OC.DatLiberOpComerRealizado-  bd_combined_OC.DatConclusaoTorresRealizado
calculos_OC_por_UG["OT_OC"] = bd_combined_OC.DatLiberOpComerRealizado-  bd_combined_OC.DatLiberacaoSFGTeste
calculos_OC_por_UG["Atraso"] =bd_combined_OC.DatLiberOpComerRealizado - bd_combined_OC.DatUGInicioOpComerOutorgado
calculos_OC_por_UG.sample(10)

,IdeUsinaOutorga,SigTipoGeracao,IO_OC,CC_OC,ME_OC,MT_OC,OT_OC,Atraso
3431,35080,UTE,-1643 days,NaT,-1663 days,NaT,85 days,-2324 days
1732,31762,EOL,368 days,307 days,48 days,48 days,21 days,-228 days
1534,31427,EOL,630 days,366 days,77 days,77 days,13 days,-18 days
1957,33139,EOL,369 days,344 days,116 days,116 days,21 days,-104 days
1305,30897,EOL,405 days,328 days,177 days,177 days,69 days,-387 days
1440,31335,EOL,274 days,274 days,150 days,150 days,45 days,104 days
1488,31362,EOL,459 days,429 days,221 days,221 days,72 days,108 days
3339,34645,EOL,657 days,449 days,187 days,25 days,69 days,-102 days
4213,38140,EOL,278 days,186 days,114 days,44 days,43 days,-74 days
585,29634,UTE,819 days,NaT,NaT,NaT,217 days,755 days


In [51]:
calculos_OC_por_UG

,IdeUsinaOutorga,SigTipoGeracao,IO_OC,CC_OC,ME_OC,MT_OC,OT_OC,Atraso
0,12_0,UHE,1532 days,855 days,346 days,NaT,8 days,665 days
1,12_1,UHE,1609 days,932 days,402 days,NaT,12 days,192 days
2,19,UTE,581 days,NaT,NaT,NaT,6 days,2299 days
3,42_0,UHE,1521 days,NaT,NaT,NaT,10 days,608 days
4,42_1,UHE,1553 days,NaT,NaT,NaT,11 days,0 days
...,...,...,...,...,...,...,...,...
5752,50872,UTE,98 days,NaT,98 days,NaT,41 days,-920 days
5776,51444,UTE,99 days,NaT,99 days,NaT,22 days,-385 days
5782,51450,UTE,104 days,NaT,104 days,NaT,26 days,-380 days
5810,51729,UTE,264 days,NaT,232 days,NaT,14 days,-768 days


In [52]:
is_first = True 
combinacoes = list(combinations([x for x in calculos_OC_por_UG.columns if x not in ["IdeUsinaOutorga","SigTipoGeracao","Atraso"]],2))
#combinacoes = list(combinations(["IO_OC","CC_OC","ME_OC","MT_OC","OT_OC"],2))

for p_marco, s_marco in combinacoes:
    idx_loop =  ((calculos_OC_por_UG[p_marco] >= calculos_OC_por_UG[s_marco]) | 
                 calculos_OC_por_UG[p_marco].isna() | 
                 calculos_OC_por_UG[s_marco].isna())

    idx = idx_loop if is_first else (idx & idx_loop)
    is_first = False 

calculos_OC_por_UG_filtered = calculos_OC_por_UG[idx].copy(deep=True)
calculos_OC_por_UG_filtered

,IdeUsinaOutorga,SigTipoGeracao,IO_OC,CC_OC,ME_OC,MT_OC,OT_OC,Atraso
0,12_0,UHE,1532 days,855 days,346 days,NaT,8 days,665 days
1,12_1,UHE,1609 days,932 days,402 days,NaT,12 days,192 days
2,19,UTE,581 days,NaT,NaT,NaT,6 days,2299 days
3,42_0,UHE,1521 days,NaT,NaT,NaT,10 days,608 days
4,42_1,UHE,1553 days,NaT,NaT,NaT,11 days,0 days
...,...,...,...,...,...,...,...,...
5752,50872,UTE,98 days,NaT,98 days,NaT,41 days,-920 days
5776,51444,UTE,99 days,NaT,99 days,NaT,22 days,-385 days
5782,51450,UTE,104 days,NaT,104 days,NaT,26 days,-380 days
5810,51729,UTE,264 days,NaT,232 days,NaT,14 days,-768 days


In [53]:
calculos_OC_por_UG_filtered = calculos_OC_por_UG.loc[calculos_OC_por_UG.groupby(["IdeUsinaOutorga"],sort=False).Atraso.idxmin()]
calculos_OC_por_UG_filtered

,IdeUsinaOutorga,SigTipoGeracao,IO_OC,CC_OC,ME_OC,MT_OC,OT_OC,Atraso
0,12_0,UHE,1532 days,855 days,346 days,NaT,8 days,665 days
1,12_1,UHE,1609 days,932 days,402 days,NaT,12 days,192 days
2,19,UTE,581 days,NaT,NaT,NaT,6 days,2299 days
3,42_0,UHE,1521 days,NaT,NaT,NaT,10 days,608 days
4,42_1,UHE,1553 days,NaT,NaT,NaT,11 days,0 days
...,...,...,...,...,...,...,...,...
5752,50872,UTE,98 days,NaT,98 days,NaT,41 days,-920 days
5776,51444,UTE,99 days,NaT,99 days,NaT,22 days,-385 days
5782,51450,UTE,104 days,NaT,104 days,NaT,26 days,-380 days
5810,51729,UTE,264 days,NaT,232 days,NaT,14 days,-768 days


In [9]:
calculos_OC_por_UG_filtered

,IdeUsinaOutorga,SigTipoGeracao,IO_OC,CC_OC,ME_OC,MT_OC,OT_OC,Atraso
0,12_0,UHE,1532 days,855 days,346 days,NaT,8 days,665 days
1,12_1,UHE,1609 days,932 days,402 days,NaT,12 days,192 days
2,19,UTE,581 days,NaT,NaT,NaT,6 days,2299 days
3,42_0,UHE,1521 days,NaT,NaT,NaT,10 days,608 days
4,42_1,UHE,1553 days,NaT,NaT,NaT,11 days,0 days
...,...,...,...,...,...,...,...,...
5752,50872,UTE,98 days,NaT,98 days,NaT,41 days,-920 days
5776,51444,UTE,99 days,NaT,99 days,NaT,22 days,-385 days
5782,51450,UTE,104 days,NaT,104 days,NaT,26 days,-380 days
5810,51729,UTE,264 days,NaT,232 days,NaT,14 days,-768 days


In [59]:
calculos_OC_por_usina = calculos_OC_por_UG.loc[calculos_OC_por_UG.groupby(["IdeUsinaOutorga"],sort=False).Atraso.idxmin()]
calculos_OC_por_usina

,IdeUsinaOutorga,SigTipoGeracao,IO_OC,CC_OC,ME_OC,MT_OC,OT_OC,Atraso
0,12_0,UHE,1532 days,855 days,346 days,NaT,8 days,665 days
1,12_1,UHE,1609 days,932 days,402 days,NaT,12 days,192 days
2,19,UTE,581 days,NaT,NaT,NaT,6 days,2299 days
3,42_0,UHE,1521 days,NaT,NaT,NaT,10 days,608 days
4,42_1,UHE,1553 days,NaT,NaT,NaT,11 days,0 days
...,...,...,...,...,...,...,...,...
5752,50872,UTE,98 days,NaT,98 days,NaT,41 days,-920 days
5776,51444,UTE,99 days,NaT,99 days,NaT,22 days,-385 days
5782,51450,UTE,104 days,NaT,104 days,NaT,26 days,-380 days
5810,51729,UTE,264 days,NaT,232 days,NaT,14 days,-768 days


In [58]:
# Cria tabelas para cálculos
calculos_OC_por_usina = bd_combined_OC[["IdeUsinaOutorga","SigTipoGeracao"]].drop_duplicates(subset="IdeUsinaOutorga",ignore_index=True).copy(deep=True)
# Calcula o atraso por usina
colunas_atrasos_usina = []
for coluna in calculos_OC_por_UG_filtered.columns:
    nome_coluna = f"{coluna}_usina"
    if coluna not in ["IdeUsinaOutorga","SigTipoGeracao"]:
        colunas_atrasos_usina.append(nome_coluna)
        calculos_OC_por_usina[nome_coluna] = calculos_OC_por_usina.IdeUsinaOutorga.apply(lambda x : calculos_OC_por_UG_filtered[calculos_OC_por_UG_filtered.IdeUsinaOutorga==x][coluna].min())
calculos_OC_por_usina 

,IdeUsinaOutorga,SigTipoGeracao,IO_OC_usina,CC_OC_usina,ME_OC_usina,MT_OC_usina,OT_OC_usina,Atraso_usina
0,12_0,UHE,1532 days,855 days,346 days,NaT,8 days,665 days
1,12_1,UHE,1609 days,932 days,402 days,NaT,12 days,192 days
2,19,UTE,581 days,NaT,NaT,NaT,6 days,2299 days
3,42_0,UHE,1521 days,NaT,NaT,NaT,10 days,608 days
4,42_1,UHE,1553 days,NaT,NaT,NaT,11 days,0 days
...,...,...,...,...,...,...,...,...
1862,50872,UTE,98 days,NaT,98 days,NaT,41 days,-920 days
1863,51444,UTE,99 days,NaT,99 days,NaT,22 days,-385 days
1864,51450,UTE,104 days,NaT,104 days,NaT,26 days,-380 days
1865,51729,UTE,264 days,NaT,232 days,NaT,14 days,-768 days


In [34]:
teste.loc[teste.IdeUsinaOutorga=="42_1"]

,IdeUsinaOutorga,SigTipoGeracao,IO_OC,CC_OC,ME_OC,MT_OC,OT_OC,Atraso
4,42_1,UHE,1553 days,NaT,NaT,NaT,11 days,0 days


In [14]:
teste

IdeUsinaOutorga  IO_OC    
1146_0           1527 days     0 days
1146_1           1602 days     0 days
1146_2           1543 days     0 days
1147             767 days    645 days
                 772 days    650 days
                               ...   
912              722 days     63 days
984_0            2068 days     0 days
984_1            2191 days    -1 days
984_2            2356 days     0 days
984_3            2806 days     0 days
Name: Atraso, Length: 2064, dtype: timedelta64[ns]

In [33]:
bd_combined

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,UFUsina,VlrPotenciaOutorgada,DatInicioObraOutorgado,DatInicioObraRealizado,DatConcretagemOutorgado,...,DatConclusaoTorresRealizado,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,SigTipoGeracao,Quant_UG,Pot_UG,origem,DatOutorgaUsina
0,12_0,2008-12-17,2007-03-01,2008-12-17,2008-12-25,RS,100.00,NaT,2004-10-15,2005-02-01,...,NaT,NaT,2008-10-15,NaT,NaT,UHE,1,50.00,bdworgs,None
1,12_1,2009-02-28,2008-09-01,2009-02-28,2009-03-12,RS,100.00,NaT,2004-10-15,2005-02-01,...,NaT,NaT,2008-10-15,NaT,NaT,UHE,1,50.00,bdworgs,None
2,19,2001-12-03,2002-01-01,2008-04-12,2008-04-18,MG,102.89,NaT,2006-09-15,NaT,...,NaT,NaT,NaT,NaT,NaT,UTE,2,36.55,bdworgs,None
3,42_0,2005-07-20,2003-11-30,2005-07-20,2005-07-30,ES,612.10,NaT,2001-05-31,2002-01-31,...,NaT,NaT,NaT,NaT,NaT,UHE,1,102.50,bdworgs,None
4,42_1,2005-08-20,2005-08-31,2005-08-20,2005-08-31,ES,612.10,NaT,2001-05-31,2002-01-31,...,NaT,NaT,NaT,NaT,NaT,UHE,1,205.00,bdworgs,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5970,56350,2022-04-27,2022-04-30,NaT,NaT,ES,37480.00,2021-12-01,2021-12-20,NaT,...,NaT,NaT,NaT,2022-02-25,2021-10-25,UTE,1,9370.00,skate,2021-11-05
5971,56350,2022-04-28,2022-05-01,NaT,NaT,ES,37480.00,2021-12-01,2021-12-20,NaT,...,NaT,NaT,NaT,2022-02-25,2021-10-25,UTE,1,9370.00,skate,2021-11-05
5972,56350,2022-04-29,2022-05-01,NaT,NaT,ES,37480.00,2021-12-01,2021-12-20,NaT,...,NaT,NaT,NaT,2022-02-25,2021-10-25,UTE,1,9370.00,skate,2021-11-05
5973,56665,2023-03-30,2023-05-30,NaT,NaT,BA,9360.00,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,UTE,1,9360.00,skate,2022-04-12


In [ ]:
# Cria tabela para calculos por geracao
calculos_por_geracao = pd.DataFrame()
calculos_por_geracao["SigTipoGeracao"] = bd_combined.SigTipoGeracao.unique() 


# Realiza calculos por geracao
for coluna in calculos_OC_por_usina.columns:
    if coluna not in ["IdeUsinaOutorga","SigTipoGeracao"]:
        calculos_por_geracao[f"{coluna[:-6]}_medio"] = calculos_por_geracao.SigTipoGeracao.apply(lambda x : calculos_OC_por_usina[calculos_OC_por_usina.SigTipoGeracao==x][coluna].mean())
        calculos_por_geracao[f"{coluna[:-6]}_std"] = calculos_por_geracao.SigTipoGeracao.apply(lambda x : calculos_OC_por_usina[calculos_OC_por_usina.SigTipoGeracao==x][coluna].std())
display(calculos_por_geracao.set_index("SigTipoGeracao").T)


In [ ]:
plot_created = False

for k,origem in enumerate(["skate","bdworgs","Ambos"]):



    # Calcula o atraso medio por usina
    colunas_atrasos_medio = []
    for coluna in calculos_OC_por_UG.columns:
        nome_coluna = f"{coluna}_medio"
        if coluna not in ["IdeUsinaOutorga","SigTipoGeracao"]:
            colunas_atrasos_medio.append(nome_coluna)
            calculos_OC_por_usina[nome_coluna] = calculos_OC_por_usina.IdeUsinaOutorga.apply(lambda x : calculos_OC_por_UG[calculos_OC_por_UG.IdeUsinaOutorga==x][coluna].mean())





    if not plot_created:    
        num_subplots = 6*(len(colunas_atrasos_medio)-1)
        fig, axes = plt.subplots(num_subplots,3,figsize=(24,num_subplots*5))
        plot_created =True
    count = -1
    for i,SigTipoGeracao in enumerate(calculos_por_geracao.SigTipoGeracao):
        for nome_coluna in colunas_atrasos_medio:
            if nome_coluna == "Atraso_OC_medio":
                continue
            count+=1
            try :
                ordered_notna = calculos_OC_por_usina[(calculos_OC_por_usina[nome_coluna].notna())
                                                 & (calculos_OC_por_usina.SigTipoGeracao == SigTipoGeracao) ].sort_values(by = nome_coluna)
                x = ordered_notna[nome_coluna].dt.days.to_numpy().reshape(-1, 1)/30
                y = ordered_notna["Atraso_OC_medio"].dt.days.to_numpy().reshape(-1, 1)/30
            
                res = stats.linregress(x.flatten(), y.flatten())
                a, b, r, p_value, std_err = res
                ax = axes[count,k]
                ax.scatter(x,y,color="blue")
                ax.set_xlabel(f"{nome_coluna} (meses)")
                ax.set_ylabel(f"Atraso_OC_medio (meses)")
                ax.plot(x,a*x+b+1,color="red",label= f"{a:.1f}x+{b:.1f}")
                ax.set_title(f"\nTipo de geração: {SigTipoGeracao}   -  $R^2$ = {100*r**2:.1f}%\n Número de usinas: {x.shape[0]} \nOrigem: {origem}")
                ax.legend(loc="lower right")
                ax.grid()
            except (ValueError,AttributeError) as error:
                print(rf"Não foi possível calcular: {nome_coluna} para {SigTipoGeracao}")

plt.tight_layout()
plt.savefig("calculos_atrasos.pdf")